In [11]:
# ==============================
# PART 2 — CHRONOLOGICAL SPLIT
# ==============================

# Required columns
FEATURES = ['OPEN', 'HIGH', 'LOW', 'CLOSE', 'TICKVOL']
TARGET = 'Label'

# Total number of rows
n_total = len(df_model)

# Split ratios
train_ratio = 0.70
val_ratio   = 0.15
test_ratio  = 0.15

# Calculate split indices (chronological)
train_end = int(n_total * train_ratio)
val_end   = train_end + int(n_total * val_ratio)

# Chronological splits (NO SHUFFLING)
df_train = df_model.iloc[:train_end].copy()
df_val   = df_model.iloc[train_end:val_end].copy()
df_test  = df_model.iloc[val_end:].copy()

# Separate features and labels (still unscaled)
X_train_raw = df_train[FEATURES]
y_train     = df_train[TARGET]

X_val_raw   = df_val[FEATURES]
y_val       = df_val[TARGET]

X_test_raw  = df_test[FEATURES]
y_test      = df_test[TARGET]



Chronological split completed.

Train set:
  X_train_raw shape: (102449, 5)
  y_train shape:     (102449,)
Label
0    0.98692
2    0.00654
1    0.00654
Name: proportion, dtype: float64 

Validation set:
  X_val_raw shape:   (21953, 5)
  y_val shape:       (21953,)
Label
0    0.981551
2    0.009247
1    0.009201
Name: proportion, dtype: float64 

Test set:
  X_test_raw shape:  (21954, 5)
  y_test shape:      (21954,)
Label
0    0.9826
1    0.0087
2    0.0087
Name: proportion, dtype: float64 

Objects ready for PART 3 (Scaling):
  X_train_raw, X_val_raw, X_test_raw
  y_train, y_val, y_test


In [12]:
# ============================================================
# PART 3 — Scaling using TRAINING DATA ONLY
# ============================================================

# ------------------------------------------------------------
# Initialize scaler
# ------------------------------------------------------------
scaler = StandardScaler()

# ------------------------------------------------------------
# FIT on TRAINING DATA ONLY
# ------------------------------------------------------------
X_train_scaled = scaler.fit_transform(X_train_raw)

# ------------------------------------------------------------
# TRANSFORM validation and test data
# ------------------------------------------------------------
X_val_scaled  = scaler.transform(X_val_raw)
X_test_scaled = scaler.transform(X_test_raw)

# ------------------------------------------------------------
# Convert to float32 (recommended for TensorFlow)
# ------------------------------------------------------------
X_train_scaled = X_train_scaled.astype(np.float32)
X_val_scaled   = X_val_scaled.astype(np.float32)
X_test_scaled  = X_test_scaled.astype(np.float32)

# ------------------------------------------------------------
# Labels remain unchanged
# ------------------------------------------------------------
y_train_scaled = y_train.values.astype(np.int32)
y_val_scaled   = y_val.values.astype(np.int32)
y_test_scaled  = y_test.values.astype(np.int32)


Objects ready for PART 4 (Create sequences):

X_train_scaled shape: (102449, 5)
y_train_scaled shape: (102449,)
X_val_scaled shape:   (21953, 5)
y_val_scaled shape:   (21953,)
X_test_scaled shape:  (21954, 5)
y_test_scaled shape:  (21954,)

Scaler fitted on TRAINING DATA ONLY.


In [13]:
# ============================
# Hyperparameters
# ============================
WINDOW_SIZE = 120
FORECAST_HORIZON = 5


# ============================
# Sequence creation function
# ============================
def create_sequences(X, y, window_size, horizon):
    """
    X: 2D array (n_samples, n_features)
    y: 1D array (n_samples,)

    Returns:
        X_seq: (num_sequences, window_size, n_features)
        y_seq: (num_sequences, horizon)
    """
    X_sequences = []
    y_sequences = []

    max_start = len(X) - window_size - horizon + 1

    for i in range(max_start):
        X_sequences.append(X[i : i + window_size])
        y_sequences.append(y[i + window_size : i + window_size + horizon])

    return np.array(X_sequences), np.array(y_sequences)


# ============================
# Create sequences
# ============================
X_train_seq, y_train_seq = create_sequences(
    X_train_scaled,
    y_train_scaled,
    WINDOW_SIZE,
    FORECAST_HORIZON
)

X_val_seq, y_val_seq = create_sequences(
    X_val_scaled,
    y_val_scaled,
    WINDOW_SIZE,
    FORECAST_HORIZON
)

X_test_seq, y_test_seq = create_sequences(
    X_test_scaled,
    y_test_scaled,
    WINDOW_SIZE,
    FORECAST_HORIZON
)



=== SEQUENCE SHAPES ===
X_train_seq: (102325, 120, 5)
y_train_seq: (102325, 5)
X_val_seq:   (21829, 120, 5)
y_val_seq:   (21829, 5)
X_test_seq:  (21830, 120, 5)
y_test_seq:  (21830, 5)

=== LABEL DISTRIBUTION (TRAIN SEQUENCES, FLATTENED) ===
Class 0: 504935
Class 1: 3345
Class 2: 3345


In [14]:
# ============================================================
# PART 5 — IMBALANCE HANDLING (CLASS-WEIGHTED LOSS ONLY)
# ============================================================

from sklearn.utils.class_weight import compute_class_weight

# ------------------------------------------------------------
# Flatten y_train_seq to compute class weights
# Shape before: (num_sequences, FORECAST_HORIZON)
# Shape after:  (num_sequences * FORECAST_HORIZON,)
# ------------------------------------------------------------
y_train_flat = y_train_seq.reshape(-1)

# ------------------------------------------------------------
# Compute class weights using training labels ONLY
# ------------------------------------------------------------
classes = np.unique(y_train_flat)

weights = compute_class_weight(
    class_weight='balanced',
    classes=classes,
    y=y_train_flat
)

# ------------------------------------------------------------
# Create Keras-compatible class_weights dictionary
# ------------------------------------------------------------
class_weights = {int(cls): float(wt) for cls, wt in zip(classes, weights)}


=== CLASS WEIGHTS (FROM TRAIN DATA ONLY) ===
Class 0: 0.3377
Class 1: 50.9841
Class 2: 50.9841

=== VARIABLES AVAILABLE FOR PART 6 ===
class_weights: {0: 0.33774974336630786, 1: 50.98405580468361, 2: 50.98405580468361}


In [15]:
# ============================
# Model configuration
# ============================
NUM_FEATURES = len(FEATURES)
NUM_CLASSES = 3
LR = 1e-3

# ============================
# Build model
# ============================
inputs = Input(shape=(WINDOW_SIZE, NUM_FEATURES))

x = LSTM(128, return_sequences=True)(inputs)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = LSTM(64, return_sequences=False)(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

# Project to multi-step horizon
from tensorflow.keras.layers import Reshape

x = Dense(FORECAST_HORIZON * NUM_CLASSES)(x)
outputs = Reshape((FORECAST_HORIZON, NUM_CLASSES))(x)
outputs = tf.keras.layers.Activation("softmax")(outputs)


model = Model(inputs, outputs)

In [16]:
# Build weighted loss function
#  class_weight is incompatible with sequence (multi-step) targets.
#  A custom loss function is required for time-distributed classification.
#  You must implement a weighted loss function to handle class imbalance.

class_weights_tensor = tf.constant(
    [class_weights[0], class_weights[1], class_weights[2]],
    dtype=tf.float32
)

def weighted_sparse_categorical_crossentropy(y_true, y_pred):
    """
    y_true: (batch, horizon)
    y_pred: (batch, horizon, num_classes)
    """
    # Standard sparse CCE per timestep
    scce = tf.keras.losses.sparse_categorical_crossentropy(
        y_true,
        y_pred
    )  # (batch, horizon)

    # Get weights for each true class
    weights = tf.gather(class_weights_tensor, tf.cast(y_true, tf.int32))
    # (batch, horizon)

    weighted_loss = scce * weights
    return tf.reduce_mean(weighted_loss)


In [17]:
# ============================
# Compile
# ============================
model.compile(
    optimizer=Adam(learning_rate=LR),
    loss=weighted_sparse_categorical_crossentropy,
    metrics=[
        tf.keras.metrics.SparseCategoricalAccuracy(name="accuracy")
    ]
)

model.summary()

# ============================
# Callbacks
# ============================
callbacks = [
    EarlyStopping(
        monitor="val_loss",
        patience=10,
        restore_best_weights=True
    ),
    ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.5,
        patience=5,
        min_lr=1e-6
    )
]

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 120, 5)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 120, 128)       │        68,608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 120, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 120, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 15)             │           975 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 5, 3)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 5, 3)           │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 119,759 (467.81 KB)

 Trainable params: 119,375 (466.31 KB)

 Non-trainable params: 384 (1.50 KB)

In [18]:
# ============================
# Train
# ============================
history = model.fit(
    X_train_seq,
    y_train_seq,
    validation_data=(X_val_seq, y_val_seq),
    epochs=200,
    batch_size=256,
    callbacks=callbacks,
    verbose=1
)


Epoch 1/200
400/400 ━━━━━━━━━━━━━━━━━━━━ 103s 253ms/step - accuracy: 0.3700 - loss: 1.3034 - val_accuracy: 0.3012 - val_loss: 1.3960 - learning_rate: 0.0010
Epoch 2/200
400/400 ━━━━━━━━━━━━━━━━━━━━ 101s 254ms/step - accuracy: 0.3764 - loss: 1.1819 - val_accuracy: 0.1045 - val_loss: 1.3929 - learning_rate: 0.0010
Epoch 3/200
400/400 ━━━━━━━━━━━━━━━━━━━━ 102s 254ms/step - accuracy: 0.3739 - loss: 1.1281 - val_accuracy: 0.3208 - val_loss: 1.3645 - learning_rate: 0.0010
Epoch 4/200
400/400 ━━━━━━━━━━━━━━━━━━━━ 103s 257ms/step - accuracy: 0.3921 - loss: 1.1224 - val_accuracy: 0.1340 - val_loss: 1.3833 - learning_rate: 0.0010
Epoch 5/200
400/400 ━━━━━━━━━━━━━━━━━━━━ 105s 263ms/step - accuracy: 0.4134 - loss: 1.1038 - val_accuracy: 0.1049 - val_loss: 1.3648 - learning_rate: 0.0010
Epoch 6/200
400/400 ━━━━━━━━━━━━━━━━━━━━ 104s 261ms/step - accuracy: 0.3855 - loss: 1.0818 - val_accuracy: 0.2468 - val_loss: 1.3422 - learning_rate: 0.0010
Epoch 7/200
400/400 ━━━━━━━━━━━━━━━━━━━━ 106s 265ms/step -

In [19]:
from sklearn.metrics import classification_report, confusion_matrix

# ============================
# Evaluate on test set
# ============================
test_loss, test_acc = model.evaluate(X_test_seq, y_test_seq, verbose=0)

print("\n=== TEST RESULTS ===")
print(f"Test Loss     : {test_loss:.6f}")
print(f"Test Accuracy : {test_acc:.6f}")

# ============================
# Detailed evaluation
# ============================
y_pred_prob = model.predict(X_test_seq)
y_pred = np.argmax(y_pred_prob, axis=-1)

# Convert y_test_seq safely to numpy
y_test_np = (
    y_test_seq.to_numpy()
    if hasattr(y_test_seq, "to_numpy")
    else np.asarray(y_test_seq)
)

# Flatten horizon dimension
y_test_flat = y_test_np.reshape(-1)
y_pred_flat = y_pred.reshape(-1)

print("\n=== CLASSIFICATION REPORT (Flattened Horizon) ===")
print(classification_report(
    y_test_flat,
    y_pred_flat,
    labels=[0,1,2],
    target_names=["No Signal", "Buy Reversal", "Sell Reversal"],
    digits=4
))

print("\n=== CONFUSION MATRIX ===")
print(confusion_matrix(y_test_flat, y_pred_flat))





=== TEST RESULTS ===
Test Loss     : 0.911882
Test Accuracy : 0.291324
683/683 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step

=== CLASSIFICATION REPORT (Flattened Horizon) ===
               precision    recall  f1-score   support

    No Signal     0.9958    0.2818    0.4393    107254
 Buy Reversal     0.0204    0.8074    0.0398       950
Sell Reversal     0.0196    0.8552    0.0384       946

     accuracy                         0.2913    109150
    macro avg     0.3453    0.6481    0.1725    109150
 weighted avg     0.9788    0.2913    0.4323    109150


=== CONFUSION MATRIX ===
[[30222 36776 40256]
 [   65   767   118]
 [   63    74   809]]
